# Our modeling approach

- We fully characterize the existence of a 1-round protocol by a Mixed Integer Linear Program.


In [6]:
from sage.all import *
import itertools
import random
from sage.sat.solvers.satsolver import SAT
from sage.sat.solvers.cryptominisat import CryptoMiniSat
from sage.misc.temporary_file import atomic_write
import copy
import time

In [127]:
solver = SAT(solver="LP")
solver.add_clause((-1,2))
solver.add_clause((1,3))
solution = solver()
print ' solution =',solution
lst = ['play', 'ground']
print 'ground' in lst

m = matrix(QQ, [[1,2],[4,2],[11,3]])
print 'm = ',m
x = []
for v in m:
    x.append(v)
    
x = matrix(QQ, x)
print 'x = ',x
 
pr = Permutations(20).random_element()
print pr, pr[4], pr[7]


v = vector(QQ, [2,3,4])
print 'len = ',len(v)

min(5, 44)

l = [1,2,3,4,5]
print l [2:4]
(5 == 5) + 0


 solution = [None, False, False, True]
True
m =  [ 1  2]
[ 4  2]
[11  3]
x =  [ 1  2]
[ 4  2]
[11  3]
[8, 14, 10, 1, 2, 9, 17, 19, 4, 6, 3, 12, 13, 5, 7, 11, 16, 15, 20, 18] 2 19
len =  3
[3, 4]


1

In [8]:
# TODO: check correctness etc
def get_at(tup, a, l):
    """Given 2*l elements flattened into a tuple, pick 1 out of each pair according to array of bits."""
    assert len(tup) == 2*l
    assert len(a) == l
    for bit in a:
        assert bit in (0, 1)

    # tup structure is: $(t_{0,0}, t_{0,1}, t_{1,0}, t_{1,1}, t_{2,0}, t_{2,1})$
    return tuple(tup[2*i + a[i]] for i in range(l))

assert get_at((3, 13, 85, 95), (0, 1), 2) == (3, 95)

In [274]:
 
def tuples_fixed_proj(l, proj_ind, proj_val):
    tuples = list()
    x = [0 for i in range(2*l)]
    for i in range(l):
        x[i*2 + proj_ind[i]] = proj_val[i]
    for g in itertools.product(range(2), repeat = l):
        for i in range(l):
            x[i*2 + 1 - proj_ind[i]] = g[i]
        tuples.append(tuple(x[:]))
    return tuples     

                               
# TODO: change the name to something more informative.
# Hack #2 from correctness: For every i,a,b with p^i_a > 0, there exists g so that (i,a,g,b) > 0.
# This requires some trial and error. Otherwise, one xi at least remains without TO-input options
# This will eventually not be part of the client constraints. Currently used to `direct' the soutions.
    
def add_a_balance(solver, p0, q1, strength = 0):
    if strength == -1:
        return
    
    if strength == 1:    
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                solver.add_constraint(sum(p0[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) +
                                      sum(-p0[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l)) 
                                      <= pow(2, l) - 1)
                solver.add_constraint(sum(p0[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) +
                                      sum(-p0[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l)) 
                                      >= 1 - pow(2, l))

    # Much weaker, but no integers..
    elif strength == 0:
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                solver.add_constraint(sum(q1[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) 
                                      <= pow(2, l) - my_eps)
                solver.add_constraint(sum(q1[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l))
                                      <= pow(2, l) - my_eps)


# Another hack from correctness. For all b, there exists at least one value of t, so 
# that every a reading this g will output b. Thus, the resulting sum of probabilities for
# that g is 1. There may be additional g's (the g results from some V assigned to server for each output
# value).                
def add_b_balance(solver, q1, strength = 0):
    if strength == -1:
        return 
    
    if strength == 0:
        for i in range(3):                              
            for b in range(2):
                # this is a relaxation - allows for splitting among several g's
                solver.add_constraint(sum(q1[(i, a, g, b)] for a in itertools.product(range(2), repeat = l) 
                                      for g in itertools.product(range(2), repeat = l)) >= 1)                


def get_rand_tup(len):
    return tuple([sage.misc.prandom.choice(range(2)) for i in range(len)])
                

# i = -1 , search for an index
# i = -2 : auxiliary for relaxed mode
# [1,0,0]
# [0,1,0]
# [0,0,1]
# [0,0,0]

def safe_insert(val_l, ind_l, (i,ind), v):
    to_search = []
    if i >= 0:
        to_search = [i]
    elif i == -1:
        to_search = [j for j in range(3)]
    elif i == -2:
        val_l[ind_l.index(ind)] = v
        return
    
    for j in to_search:
        if (j,ind) in ind_l:
            val_l[ind_l.index((j,ind))] = v


def find_set_server_vars(l, p0):
    try:
  #      print 'How much is server variable space limited by this solution & correctness?'
  #      print '|List1| = ',len(p0)
  #      print 'List=',p0
        p1 = []
        for i in range(3):                              
            for t in itertools.product(range(2), repeat = 2*l):
                p1.append((i,t))

        for (y,(a,g,b)) in p0:
            for x in range(3):
                if ((b == 0) and (y == x)) or ((b == 1) and (y != x)):
                    all = tuples_fixed_proj(l, a, g)
                    for t in all:
                        try:
                #            print 'removing ',(x, t)
                #            print 'Due to ',(y, (a,g,b))
                            p1.remove((x, t))
                        except Exception as e:
                            pass

        print 'number of remaining server values =',len(p1)
        if (len(p1) > 0):
            print 'server variables are ',p1
    except: 'Uncaught exception!'    


def permute_rows(m, b):
    
    nrows = m.nrows()
    perm = Permutations(nrows).random_element()
    pm = [row for row in m]
    pb = [v for v in b]
    
    new_m = matrix(QQ, [pm[perm[i] - 1] for i in range(nrows)])
    
    new_b = vector(QQ, [pb[perm[i] - 1] for i in range(nrows)])
    
    return (new_m, new_b)
    
    
    
def get_full_rank_sub(m, b):
   
    nrows = m.nrows()
    new_mat = []
    new_b = []
    
 #   print 'into get_full_rank_sub',m.nrows(), m.ncols()
    
    for (i,row) in enumerate(m):
        so_far = (matrix(QQ, new_mat)).transpose()
        try:
            so_far\row
        except Exception as e:
#            print 'at full rank sub Exception = ',e,i
            new_mat.append(row)
            new_b.append(b[i])
        
    new_mat = matrix(QQ, new_mat)
    new_b = vector(new_b)
    
    return (new_mat, new_b)


def print_matrix(m):

    if m.ncols < 37:
        print m
        return
    
    per_row = 30
    for row in m:
        n = ceil(QQ(m.nrows())/per_row)
        for i in range(n):
            ran = min(per_row, m.nrows() - i * per_row)
            print [row[i * per_row + j] for j in range(ran)]
        print '\n'    
          

def non_zeros(m):            
    return sum(sum(1 for j in row if abs(j) != 0) for row in m)        
        
        
def get_inv_sub_matrix(m, b):
    
    (new_mat, new_b) =  get_full_rank_sub(m, b)
    
    ncols = new_mat.ncols()
    print 'new_mat rank /  ',new_mat.rank(),'(',new_mat.nrows(),',',new_mat.ncols(),')'
    
    new_mat_tr = new_mat.transpose()
    # a dummy vector
    b_dum = vector(QQ, [0 for i in range(ncols)]) 
    
    (full_sub, b_dum) = permute_rows(new_mat_tr, b_dum)
    (full_sub, b_dum) = get_full_rank_sub(full_sub, b_dum)
    
    # Need to transpose full_sub back!
    full_sub = full_sub.transpose()
    
    inv_sub = full_sub.inverse()
    
    print 'x = A^{-1}*b',
 #   print full_sub
    
    det_A = full_sub.determinant()
    print '|A| = ', det_A
    # Finding independent columns
    print 'Sparsity parameters',non_zeros(full_sub),' => ',non_zeros(inv_sub)
    
    x = inv_sub * new_b
    print 'x = ',x
    print 'len(x) = ',len(x)
    
    if (full_sub.nrows() < 10 and abs(det_A) > 0) or (full_sub.ncols() < 20):
        print print_matrix(inv_sub)
    
    return x


def find_small_serv(l, out_of = 3):
    
    b = []
    constraints = []
    t_set = []
    t_part = [[],[],[]]
    
    for t in itertools.product(range(2), repeat = 2*l):
        i = sage.misc.prandom.choice(range(out_of))
        if (i < 3):
            t_set.append((i,t))
            t_part[i].append(t)
                
    for i in range(3):
            if len(len(t_parts[i])) == 0:
                print 'failed attempt'
                return -1
            
    base_len = len(t_set)
    # add auxiliary variables
    
    for a in itertools.product(range(2), repeat = l):
        for y in range(3):
            for z in range(2):
                for g in itertools.product(range(2), repeat = l):
                    t_set.append((a,y,z,g))

    
    t_len = len(t_set)
    
    # add distribution constraints
    for x in range(3):
        cur_row = [0 for i in range(t_len)]
        for (i,t) in t_set[0:base_len]:
            if (x == i):
                cur_row[t_set.index(i,t)] = 1
        constraints.append(cur_row)
        b.append(1)
    
    
    for a in itertools.product(range(2), repeat = l): 
        for y in range(3):
            cur_row = [0 for i in range(t_len)]
            for g in itertools.product(range(2), repeat = l):
                cur_row[t_set.index(a,y,1,g)] = 1
                cur_row[t_set.index(a,y,0,g)] = -1
            constraints.append(cur_row)    
            b.append(0)

    
    for a in itertools.product(range(2), repeat = l):
        cur_row = [0 for i in range(t_len)]    
        for y in range(3):
            for g in itertools.product(range(2), repeat = l):
                cur_row[t_set.index(a,y,0,g)] = 1
            constraints.append(cur_row)    
            b.append(1)    
    
    # add security against malicious clients
    for a in itertools.product(range(2), repeat = l):
        for x in range(3):
            for g in itertools.product(range(2), repeat = l):
                ts = tuples_fixed_proj(l, a, g)
                cur_row = [0 for i in range(t_len)]
                for t in ts:
                    if t in t_part[x]:
                        cur_row[ts.index((x, t))] = 1
                    for y in itertools.product(range(2), repeat = l):
                        z = 0 + (x == y)
                        cur_row[ts.index((a,y,z,g))] = -1 
                constraints.append(cur_row)
                b.append(0)
    
    m_constr = matrix(QQ,constraints)
    right_side = vector(QQ,b)
    
    print 'basic facts about the distribution: '
    'rank\ncols = ',m_constr.rank() ,m_constr.ncols()
    
    for cur_row in constraints:
        solver.add_constrtaints(sum[cur_row[j]*prot[j] for j in range(t_len)] == b[j])
        
    try:    
        solver.solve()  
        print 'solution = '
        print 'main variables = ' 
        print [solver.get_values(prot[j]) for j in range(base_len)]
        print 'aux variables = '
        print [solver.get_values(prot[j]) for j in range(base_len,t_len)]
        
        
    except Exception as e:
        print 'Exception = ',e 

# only captures a small subset of constraints                
def test_linear_solutions(l, out_of = 3, verbose = 0):
    
        b = []
        constraints = []
        
        if verbose:
            print 'generating constraints ...'
        
        
        a_set = []
        a_only_set = [[],[],[]]
        for a in itertools.product(range(2), repeat = l):
            i = sage.misc.prandom.choice(range(out_of))
            if (i < 3):
                a_only_set[i].append(a)
                for g in itertools.product(range(2), repeat = l):
                    for z in range(2):
                        a_set.append((i,(a,g,z)))
        
        for i in range(3):
            if len(a_only_set[i]) == 0:
                if verbose:
                    print 'failed attempt'
                return -1
        
        if verbose:
            print 'a_only_set = ', a_only_set
        
        ts = [get_rand_tup(2*l) for i in range(3)]
        if verbose:
            print 'ts = ', ts
        
  #      for x in range(3):
  #          for y in range(3):
  #              z = 1 - int(x == y)
  #              for a in a_only_set[y]:
  #                  if (y, (a,  get_at(ts[x], a, l), z)) in a_set:
  #                      a_set.remove((y, (a,  get_at(ts[x],a,l), z)))
                    
        a_set_len = len(a_set) 
        if verbose:
            print '|List0|=', a_set_len
        
        #insure non-triviality
        # pick 3 t's for x1, x2, x3 at random
        
        for x in range(3):
            for y in range(3):
                z = (x == y) + 0
                cur_row = [0 for i in range(a_set_len)]
                for a in a_only_set[y]:
                    if (y, (a, get_at(ts[x],a,l), z)) in a_set:
                        cur_row[a_set.index((y, (a, get_at(ts[x],a,l), z)))] = 1
                constraints.append(cur_row)
                b.append(1)
        
        # well-formed distribution
        for i in range(3):
            cur_row = [0 for j in range(a_set_len)]
            for (j, (a, g, z)) in a_set:
                if j == i:
                    cur_row[a_set.index((j, (a, g, z)))] = 1       
            constraints.append(cur_row)
            b.append(1)


        # require equiality distributions
        for i in range(3):
            for a in a_only_set[i]:
                for g in itertools.product(range(2), repeat = l): 
                    if g != zeros:
                        cur_row = [0 for i in range(a_set_len)]
                        # We are guaranteed that a is not completely out of the 
                        # set. If it was, eccidentally, this will make the set unsolvable
                        safe_insert(cur_row, a_set, (i, (a, zeros, 0)), 1)
                        safe_insert(cur_row, a_set, (i, (a, zeros, 1)), 1)
                        safe_insert(cur_row, a_set, (i, (a, g, 0)), -1)
                        safe_insert(cur_row, a_set, (i, (a, g, 1)), -1)
                        b.append(0)
                        constraints.append(cur_row)

        
        # require security against a malicious server
        
        for t in itertools.product(range(2), repeat = 2*l):
            cur_row = [0 for i in range(a_set_len)]
            b.append(1)
            for a in itertools.product(range(2), repeat = l):
                safe_insert(cur_row, a_set, (-1, (a, get_at(t,a,l), 1)), 1)
            constraints.append(cur_row)
       
                
        
        m_constr = matrix(QQ,constraints)
        right_side = vector(QQ,b)
        
        solver = MixedIntegerLinearProgram(solver='ppl')
        prot = solver.new_variable(nonnegative=True)
            
        if verbose:
            print 'Created linear system of rank',m_constr.rank(),'/',pow(2, 2*l + 1),' vs ',a_set_len
        try:
            v = m_constr\right_side
#            ker = m_constr.right_kernel()        
#            print 'solution for linear system = ',v

            print 'len(solution) = ',len(v)
            nz = sum(1 for x in v if abs(x) > 0)
            
            print 'non-zeros ', nz
            
         #   eff_len = a_set_len - expl_solvable*pow(2, 2*l)
         #   a_set = a_set[0: eff_len]
            
            # uses the fact that the auxiliary varias are at the end
            
            x = get_inv_sub_matrix(m_constr, right_side)
            
            if verbose:
                print '==================================================V'
                print 'Now looking for a well-formed solution, not just any'
            
            for (i,cur_row) in enumerate(constraints):
                solver.add_constraint(sum(cur_row[j]*prot[j] for j in range(a_set_len)) == b[i])
                
            solver.solve()  
            
            print 'A solution to the full LP exists!'
            
            sol = [solver.get_value(prot[i]) for i in range(a_set_len)]
            
            
            print 'solution =', sol
            
            for (i,val) in enumerate(a_set):
                if sol[i] == 0:
                    a_set.remove(val)
            
       #     find_set_server_vars(l, a_set)
            
            
         #   print 'solution = ',prot
                
            
            print '==================================================V'
        except Exception as e:
            print 'Exception = ',e
            if verbose:
                print '==================================================X'
            return -1
        
 #       print 'Returning ',(nz, a_set_len)
        return nz        


# global variables

my_eps = QQ(1/10000)

In [275]:
# main     
import time 


def main(dim, nsteps, lim = 3):

    report_granularity = 1
    for l in range(dim, dim+1):
        for step in range(nsteps):
            if (step % report_granularity) == 0:
                print 'STEP # ',step
                t0 = time.time()
            zeros = tuple((0 for i in range(l)))

            try:
                q = test_linear_solutions(l, lim)
                if q != -1:
                    print 'A complete solution exists! nz=',q

            finally:
                if (step % report_granularity) == 0:
                    print 'time =', time.time() - t0,'\n'



In [276]:
main (3, 100)

STEP #  0
len(solution) =  128
non-zeros  23
new_mat rank /   39 ( 39 , 128 )
x = A^{-1}*b |A| =  -1
Sparsity parameters 150  =>  233
x =  (1, -3, 1, -3, 0, -5, 1, -1, 0, 1, 1, -1, -1, 1, 0, 1, -1, -1, 2, 1, 2, 0, 0, 1, 0, 0, -1, -1, 0, 2, 1, 0, 0, 1, 1, 1, 0, 1, 1)
len(x) =  39
Exception =  PPL : There is no feasible solution
time = 1.91983795166 

STEP #  1
len(solution) =  128
non-zeros  19
new_mat rank /   39 ( 39 , 128 )
x = A^{-1}*b |A| =  1
Sparsity parameters 151  =>  325
x =  (-1, 0, 1, 0, -1, 0, 0, 0, 2, -3, 0, 1, 0, 1, 0, -7, 1, 0, 1, -1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, -1, 1, 1)
len(x) =  39
Exception =  PPL : There is no feasible solution
time = 1.75703191757 

STEP #  2
time = 0.000655174255371 

STEP #  3
len(solution) =  128
non-zeros  18
new_mat rank /   38 ( 38 , 128 )
x = A^{-1}*b |A| =  2
Sparsity parameters 163  =>  292
x =  (1, 0, 1, 1, 0, -4, 3/2, 0, 0, 0, 0, 1, -5, 1, 0, 0, 0, 0, 0, 0, -3/2, 1, 0, 0, 3/2, 0, 1, 1, 0, -1, 1, 0, -1/2, 0, 0, 1, 1, 1

len(solution) =  128
non-zeros  17
new_mat rank /   39 ( 39 , 128 )
x = A^{-1}*b |A| =  -1
Sparsity parameters 143  =>  272
x =  (1, 1, -7, 1, 1, -1, 1, 0, 0, 0, -4, 1, 1, 0, 0, 1, 0, 1, -1, 1, 2, 1, -2, 2, 1, -1, 0, 0, 0, -1, 1, 0, 0, 1, -1, 0, 1, 1, 1)
len(x) =  39
Exception =  PPL : There is no feasible solution
time = 1.62970900536 

STEP #  28
len(solution) =  128
non-zeros  17
new_mat rank /   39 ( 39 , 128 )
x = A^{-1}*b |A| =  1
Sparsity parameters 157  =>  313
x =  (-1, 1, -5, 1, -1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, -5, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, -1, 1, 0, 0, 0, -1, 1, 0, 1)
len(x) =  39
Exception =  PPL : There is no feasible solution
time = 1.67005205154 

STEP #  29
len(solution) =  128
non-zeros  21
new_mat rank /   39 ( 39 , 128 )
x = A^{-1}*b |A| =  1
Sparsity parameters 144  =>  299
x =  (0, 0, -9, -1, 1, 0, 1, 1, 1, 1, 0, -1, 0, 1, -1, 0, 1, -1, 1, 1, 1, 0, 1, 0, -1, -1, 0, 1, 1, 1, -1, 0, 1, 1, 1, 1, 0, 0, 1)
len(x) =  39
Exception =  PPL : There is no fea

new_mat rank /   39 ( 39 , 128 )
x = A^{-1}*b |A| =  1
Sparsity parameters 145  =>  227
x =  (1, -9, -1, 2, 0, -3, -1, -1, -1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, -1, 0, 1, 0, 1, 1, 1, 0)
len(x) =  39
Exception =  PPL : There is no feasible solution
time = 1.65842580795 

STEP #  54
len(solution) =  128
non-zeros  23
new_mat rank /   39 ( 39 , 128 )
x = A^{-1}*b |A| =  2
Sparsity parameters 161  =>  569
x =  (1/2, -3, -6, 1/2, -2, 1, 5/2, 1, -2, 1, -3/2, 1, 1, 1/2, -1/2, 1, 3/2, 1, 1/2, -1/2, 0, -1, 1/2, 1/2, 0, 1, 1, 3/2, 1, 3/2, 3/2, -3/2, -3/2, 1/2, -1/2, -1/2, -1, 3/2, 1)
len(x) =  39
Exception =  PPL : There is no feasible solution
time = 1.64315104485 

STEP #  55
len(solution) =  128
non-zeros  21
new_mat rank /   39 ( 39 , 128 )
x = A^{-1}*b |A| =  1
Sparsity parameters 148  =>  283
x =  (-9, 0, 1, 3, -1, 1, -1, -1, -1, 1, -1, 2, -1, 0, -2, 2, 1, 1, -1, -1, 0, 1, 1, 1, 0, 0, -1, 1, -2, 1, 0, -1, 1, 1, 1, 2, 3, 0, 1)
len(x) =  39
Exception =  PPL : 